# First clean

Removing duplicate albums based on 1) album name; 2) album id; 3) feature difference

In [2]:
setwd("/home/pa/Documents/github/doc_suomi/code")
source("utils.R")
source("data_cook.R")

`summarise()` has grouped output by 'album_id', 'name'. You can override using the `.groups` argument.



In [ ]:
new = fread("/home/pa/Documents/github/doc_suomi/data/raw_dataset/sample2.csv")
old = fread("/home/pa/Documents/github/doc_suomi/data/raw_dataset/grande_sem_low.csv")

base = dplyr::bind_rows(new, old)

In [ ]:
replicates = base %>%
    group_by(album_id, name) %>%
    summarise(overall = sum(valence, energy, loudness, tempo), name = unique(name))

replicates = replicates[order(-replicates$overall), ]

In [ ]:
#Calculate 0 differences
valores = c(replicates$overall)
album_ids = c(replicates$album_id)
nome = c(replicates$name)

difs = c()
albums = c()
nomes = c()
nomes2 = c()
for(j in 1:(length(valores)-1)){
    difs[[j]] = abs(valores[j] - valores[j+1])
    albums[[j]] = album_ids[j]
    nomes[[j]] = nome[j]
    nomes2[[j]] = nome[j+1]
    }

replicates = tibble(album_id = albums, difference =difs, nome1 = nomes, nomes2 = nomes2)
replicates <- as.data.frame(lapply(replicates, unlist))

In [ ]:
#replicates[order(replicates$difference)]
nomes_iguais = replicates[order(replicates$difference, replicates$nome1),] %>% filter(nome1 == nomes2)
nomes_iguais = c(nomes_iguais$album_id)
# replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference <= 0.005) %>% filter(difference > 0) %>%
#           filter(nome1 != nomes2) %>% head(20)

In [ ]:
dif_zero = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference == 0)
dif_zero = c(dif_zero$album_id)

In [ ]:
dif_pequena = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference <= 0.005)
dif_pequena = c(dif_pequena$album_id)

In [ ]:
filtro = unique(c(nomes_iguais, dif_zero, dif_pequena))

In [ ]:
base %>%
    filter(!album_id %in% filtro) %>% 
    dplyr::select(album_id, track_number, name, valence, energy, loudness, tempo, danceability) %>%
    write.csv("/home/pa/Documents/github/doc_suomi/data/treated_data/data.csv")

# Second clean

Removing albums with wrong number of tracks (from 1 to 10, but missing track 9)

In [ ]:
setwd("/home/pa/Documents/github/doc_suomi/data/treated_data")

base = fread("data.csv")
#If album length != largest track_number, remove album 
raw = base %>% group_by(album_id, track_number) %>%
        summarise(valence = mean(valence), energy = mean(energy), 
                  loudness = mean(loudness), tempo = mean(tempo)) %>%
        mutate(album_length = NROW(track_number)) %>% ungroup() %>%
        group_by(album_id) %>%
        filter(max(track_number) != unique(album_length))

albums_out = c(unique(raw$album_id))

base %<>% 
    filter(!album_id %in% albums_out) %>%
    write.csv("/home/pa/Documents/github/doc_suomi/data/treated_data/data.csv")

In [ ]:
fread("data.csv") %>%
    select("album_id", "name", "track_number", "valence", "energy", "loudness", "tempo", "danceability") %>%
    group_by(album_id, name) %>%
    summarise(valence = mean(valence)) %>%
    arrange("name") -> third

In [ ]:
third[!duplicated(third$name), ] %>%
    arrange(name) -> final 

fread("data.csv") %>%
    select("album_id", "name", "track_number", "valence", "energy", "loudness", "tempo", "danceability") %>%
    filter(album_id %in% final$album_id) %>% write.csv("data.csv")

In [ ]:
#filtering by name
base = fread("data.csv")
oi = base %>%
    filter(track_number == 1)

deluxe = oi[grepl("Deluxe", oi$name, ignore.case = TRUE), ]$album_id
ao_vivo = oi[grepl("ao vivo", oi$name, ignore.case = TRUE), ]$album_id
live = oi[grepl("live", oi$name, ignore.case = TRUE), ]$album_id
special = oi[grepl("special edition", oi$name, ignore.case = TRUE), ]$album_id
out = unique(c(deluxe, ao_vivo, live, special))

final = fread("data.csv")
base = final %>% filter(!album_id %in% out)
write.csv(base, "data.csv")

In [ ]:
fread("data.csv") %>% 
     select(album_id, name, track_number, valence, energy, loudness, tempo, danceability) %>%
     filter(track_number == 1) %>% arrange(name) %>% distinct(name) -> chines 


In [ ]:
fread("data.csv") %>% 
    select(album_id, name, track_number, valence, energy, loudness, tempo, danceability) %>%  
    filter(name %in% chines$name) %>% write.csv("data.csv")

# Cleaning low level

In [ ]:
low = fread("/home/pasoneto/Documents/github/doc_suomi/data/raw_dataset/grande_com_low.csv")
low %<>% select(c('danceability', 'energy', 'key', 'loudness_x', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
'liveness', 'valence', 'tempo_x', 'duration_ms', 'time_signature', 'track_number', 'album_id', 'name', 
'loudness_y', 'tempo_y', 'tempo_confidence', 'key_confidence', 'mode_confidence', 'time_signature_confidence', 'track_id'))

colnames(low) <- c('danceability', 'energy', 'key', 'loudness_overall', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
'liveness', 'valence', 'tempo_overall', 'duration_ms', 'time_signature', 'track_number', 'album_id', 'name', 
'loudness_continuous', 'tempo_continuous', 'tempo_confidence', 'key_confidence', 'mode_confidence', 'time_signature_confidence', 'track_id')

length(unique(low$album_id))

In [ ]:
replicates = low %>%
    group_by(album_id) %>%
    summarise(overall = sum(valence, loudness_overall, energy, tempo_overall), name = unique(name))

replicates = replicates[order(-replicates$overall), ]

#Calculate 0 differences
valores = c(replicates$overall)
album_ids = c(replicates$album_id)
nome = c(replicates$name)

difs = c()
albums = c()
nomes = c()
nomes2 = c()
for(j in 1:(length(valores)-1)){
    difs[[j]] = abs(valores[j] - valores[j+1])
    albums[[j]] = album_ids[j]
    nomes[[j]] = nome[j]
    nomes2[[j]] = nome[j+1]
    }

replicates = tibble(album_id = albums, difference =difs, nome1 = nomes, nomes2 = nomes2)

replicates <- as.data.frame(lapply(replicates, unlist))
nomes_iguais = replicates[order(replicates$difference, replicates$nome1),] %>% filter(nome1 == nomes2)

nomes_iguais = c(nomes_iguais$album_id)

dif_zero = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference == 0)
dif_zero = c(dif_zero$album_id)

dif_pequena = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference <= 0.005) %>% filter(!album_id %in% nomes_iguais)
dif_pequena = c(dif_pequena$album_id)

filtro = unique(c(nomes_iguais, dif_zero, dif_pequena))

low %<>%
     filter(!album_id %in% filtro) 

low %>% 
     write.csv("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data_low.csv")

In [ ]:
low = fread("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data_low.csv")

In [ ]:
low %>% 
    group_by(album_id, name) %>%
    summarise(mean(valence)) %>%
    arrange(name) -> reman

In [ ]:
low %>% 
    filter(!album_id %in% c(reman[(duplicated(reman$name)), ]$album_id)) %>%
    write.csv("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data_low.csv")

# Low - second clean

Definitivo. Tirando repetidos

In [3]:
## using previous reference to get only non-duplicate albums
base() %>% 
    filter(track_number == 1) %>% 
    select(album_id) %>% 
    unlist() %>% 
    unname() -> album_no_repetition

low_com_duplica = fread("/home/pa/Documents/github/doc_suomi/data/treated_data/data_low.csv") %>% select(!V1)

low_com_duplica %>%
    filter(album_id %in% album_no_repetition) %>%
    write.csv("/home/pa/Documents/github/doc_suomi/data/treated_data/data_low.csv")